# Accessing NDVI data on Tribal Subdivisions

Download data for the vegetation coding challenge

In [1]:
id = 'stars'
site_name = 'Gila River Indian Community'
data_dir = 'gila-river'

# STEP 1: Site map

We’ll need some Python libraries to complete this workflow.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import necessary libraries</div></div><div class="callout-body-container callout-body"><p>In the cell below, making sure to keep the packages in order, add
packages for:</p>
<ul>
<li>Working with DataFrames</li>
<li>Working with GeoDataFrames</li>
<li>Making interactive plots of tabular and vector data</li>
</ul></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>What are we using the rest of these packages for? See if you can
figure it out as you complete the notebook.</p></div></div>

In [2]:
import json
import os
import pathlib
from glob import glob

import earthpy.api.appeears as eaapp
import earthpy
import hvplot.xarray
import rioxarray as rxr
import xarray as xr

We have one more setup task. We’re not going to be able to load all our
data directly from the web to Python this time. That means we need to
set up a place for it.

> **GOTCHA ALERT!**
>
> A lot of times in Python we say “directory” to mean a “folder” on your
> computer. The two words mean the same thing in this context.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace ‘my-data-folder’ with a <strong>descriptive</strong>
directory name.</li>
<li>Run the cell to display your project directory.</li>
<li>Can you find the directory, either in a terminal or through your
operating system’s file browser/explorer/finder?</li>
</ol></div></div>

In [4]:
# Create a project directory in the system data folder
project = earthpy.project.Project(project_dirname=data_dir)
project.project_dir

## Study Area: Gila River Indian Community

### Earth Data Science data formats

In Earth Data Science, we get data in three main formats:

| Data type | Descriptions | Common file formats | Python type |
|-----|------------------------|-----------------------------------|---------|
| Time Series | The same data points (e.g. streamflow) collected multiple times over time | Tabular formats (e.g. .csv, or .xlsx) | pandas DataFrame |
| Vector | Points, lines, and areas (with coordinates) | Shapefile (often an archive like a `.zip` file because a Shapefile is actually a collection of at least 3 files) | geopandas GeoDataFrame |
| Raster | Evenly spaced spatial grid (with coordinates) | GeoTIFF (`.tif`), NetCDF (`.nc`), HDF (`.hdf`) | rioxarray DataArray |

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Check out the sections about about <a
href="https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/">vector
data</a> and <a
href="https://www.earthdatascience.org/courses/intro-to-earth-data-science/file-formats/use-spatial-data/use-raster-data/">raster
data</a> in the textbook.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>For this coding challenge, we are interested in the boundary of the
<span data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="2"></span>. In the cell below, answer the
following question: <strong>What data type do you think the boundary
will be?</strong></p></div></div>

YOUR ANSWER HERE

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ul>
<li><strong>Search</strong> the US open data website for “American
Indian Tribal Subdivisions” <a
href="https://catalog.data.gov/">data.gov</a> for</li>
<li>Find the <code>tl_2020_us_aitsn.zip</code> file. Click on
<code>Download</code>. Put the downloaded files into your data folder
for this project.</li>
<li>Modify the code below to use <strong>descriptive variable
names</strong> and to point at the path where you put your data. Feel
free to refer back to previous challenges for similar code!</li>
</ul></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-info"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div></div><div class="callout-body-container callout-body"><p>Take a look at your sample code for this task below. You might notice
that there is some extra code:</p>
<div class="sourceCode" id="cb1"><pre
class="sourceCode python"><code class="sourceCode python"><span id="cb1-1"><a href="#cb1-1" aria-hidden="true" tabindex="-1"></a><span class="co"># Create a session with a valid User-Agent header</span></span>
<span id="cb1-2"><a href="#cb1-2" aria-hidden="true" tabindex="-1"></a>fs <span class="op">=</span> fsspec.filesystem(<span class="st">&quot;http&quot;</span>, headers<span class="op">=</span>{<span class="st">&quot;User-Agent&quot;</span>: <span class="st">&quot;Mozilla/5.0&quot;</span>})</span>
<span id="cb1-3"><a href="#cb1-3" aria-hidden="true" tabindex="-1"></a></span>
<span id="cb1-4"><a href="#cb1-4" aria-hidden="true" tabindex="-1"></a><span class="co"># Open the file using fsspec and pass to geopandas</span></span>
<span id="cb1-5"><a href="#cb1-5" aria-hidden="true" tabindex="-1"></a><span class="cf">with</span> fs.<span class="bu">open</span>(url) <span class="im">as</span> f:</span>
<span id="cb1-6"><a href="#cb1-6" aria-hidden="true" tabindex="-1"></a>    gdf <span class="op">=</span> gpd.read_file(f)</span></code></pre></div>
<p>We need this code because the <code>data.gov</code> site doesn’t want
bots or code downloading their data. What we’re doing won’t cause any
harm, but computers can be programmed to repeatedly hit URLs so that
they overload the online data service. This is sometimes called a
<strong>denial of service</strong> attack. With this extra two lines of
code, your code can <strong>pretend to be the Mozilla web
browser</strong>, allowing the data to download normally.</p></div></div>

In [6]:
# Download the site boundary

# Create a session with a valid User-Agent header
fs = fsspec.filesystem("http", headers={"User-Agent": "Mozilla/5.0"})

# Open the file using fsspec and pass to geopandas
with fs.open(url) as f:
    gdf = gpd.read_file(f)
    
# Check that the data were downloaded correctly

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ul>
<li><strong>Select</strong> all the subdivision of the Gila River Indian
Community from the resulting <code>DataFrame</code></li>
<li>**Spatially merge the results to get a single boundary for the Gila
River Indian Community.</li>
</ul></div></div>

### Site Map

We usually want to create a site map as a first step when working with
geospatial data. This helps us see that we’re looking at the right
location, and learn something about the context of the analysis.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Plot your <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> shapefile on an interactive map</li>
<li>Make sure to add a title</li>
<li>Add ESRI World Imagery as the basemap/background using the
<code>tiles=...</code> parameter. To get the tiles to match up with your
data, you will also need to add the <code>geo=True</code>
parameter.</li>
</ol></div></div>

In [8]:
# Plot the site boundary